In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import gc

def getHeatMap(newdataf,metdataf,month,dirpath,count,startDate):
    #Filtraggio per mese
    newdf=newdataf[newdataf['hours'].str.contains(month)]
    metdf=metdataf[metdataf['hours'].str.contains(month)]
    #display(newdf)
    #display(metdf)
    #Creazione dataframe finale
    finaldf=pd.DataFrame(columns=['hours','bool','power_dev','power_met'])
    
    finaldf['hours']=newdf['hours']
    finaldf['bool'] = np.where(newdf.power <= metdataf.power,1,0).astype(int)
    finaldf['power_met']=metdf['power']
    finaldf['power_dev']=newdf['power']
    finaldf.to_csv('finalcsv',index=False)
    writedf=finaldf[finaldf['bool']==0]
    heatDir=os.path.join(os.path.abspath('..'),os.path.join("check\\","EB"+dirpath[17]))
    if not os.path.exists(heatDir):
        os.makedirs(heatDir)
    heatmapPath=os.path.join(heatDir,month+'.csv')
    print(heatmapPath)
    writedf.to_csv(heatmapPath,index=False)
    
    print('HO FINITO')
   
 


    

    
#Funzione che ritorna dataframe in base al mese   
def switch_demo(boolval):
    global jandf
    global metjandf
    if(boolval==False):
        return jandf
    else:
        return metjandf
#Funzione che assegna le modifiche al dataframe in base al mese
def switch_df(df,boolval):
    global jandf
    global metjandf
    if(boolval==False):
        jandf=df
    else:
        metjandf=df
def process(df,filepath,count,boolval):
    global jandf
    global metjandf
    
    df['timestamp'] = pd.to_datetime(df['timestamp'],utc=True).dt.tz_convert('Europe/Rome')
    #df['Date'] = df.timestamp.dt.tz_convert('Europe/Rome')
    #print(df)
    #df['timestamp'].apply(lambda x: pd.to_datetime(x).tz_convert('Europe/Rome'))
    
    #df['timestamp']=pd.Timestamp(df['timestamp'],tz='Europe/Rome')
    # crea una time series settando la data come index
    #df = df.set_index('timestamp')
    #Aggiungi una colonna contenente  data e ora per heatmap
    df['hours']=df['timestamp'].dt.strftime("%d-%b-%Y (%H-%M-%S)")
    #df['hours']=df.index.to_series(keep_tz=True).dt.strftime("%d-%b-%Y (%H-%M-%S)")
    df['day']=df['timestamp'].dt.strftime("%d").str.lstrip("0").replace(" 0", " ")
    df=df.set_index('timestamp')
    
    monthList=['Jan']
    #Esegui i vari filtraggi e crea un dataframe contenente i valori per ogni mese
    for month in monthList:
        newdf=df[df['hours'].str.contains(month)]
        #display(newdf)
        filterdf=switch_demo(boolval)
        #display(filterdf)
        filterdf.loc[filterdf.hours.isin(newdf.hours) , 'power'] = filterdf['power']+newdf['power']
        #print('dopo')
        #display(filterdf)
        
        switch_df(filterdf,boolval)
    del df,newdf,filterdf
    gc.collect()
        
        
        
    
 


ModuleNotFoundError: No module named 'seaborn'

In [ ]:
#DIRPATH=["\Dati puntuali EB1 Gennaio-Giugno 2019","\Dati puntuali EB3 Gennaio-Giugno 2019","\Dati puntuali EB4 Gennaio-Giugno 2019","\Dati puntuali EB5 Gennaio-Giugno 2019","\Dati puntuali EB6 Gennaio-Giugno 2019","\Dati puntuali EB7 Gennaio-Giugno 2019","\Dati puntuali EB8 Gennaio-Giugno 2019","\Dati puntuali EB9 Gennaio-Giugno 2019"]
DIRPATH=["\Dati puntuali EB4 Gennaio-Giugno 2019"]
count=0
for dirpath in DIRPATH:
    count=count+1
    print("Start "+str(count))
    dirp=os.path.join(os.path.abspath('..'),"data_rec"+dirpath+"\\")
    filelist=[f for f in os.listdir(dirp)]
    jandf = metjandf= pd.DataFrame({
        'date': pd.date_range(
            start=pd.Timestamp('2019-01'),
            end=pd.Timestamp('2019-01') + pd.offsets.MonthEnd(0) + pd.Timedelta(days=1),
            freq='S',  # <--- try '3h', '6h', '12h' if you want
            closed='left',
            tz='Europe/Rome'
                )
        })
    pd.set_option('display.max_rows', 10000)
    #display(jandf)
    
    jandf = jandf.set_index('date')
    metjandf = metjandf.set_index('date')
    
    jandf['hours']=jandf.index.to_series(keep_tz=True).dt.strftime("%d-%b-%Y (%H-%M-%S)")
    
    metjandf['hours']=metjandf.index.to_series(keep_tz=True).dt.strftime("%d-%b-%Y (%H-%M-%S)")

    jandf['power']=metjandf['power']=0
    for filepath in filelist:
        dataPath=os.path.join(dirp,filepath)
        print(filepath)
        if("Meter" in filepath):
            boolval=True
        else:
            boolval=False
        for chunck in pd.read_csv(dataPath, sep=',',chunksize=500000):
            process(chunck,filepath,count,boolval)
            print('Fine')
            print('-----')
    
    #Creazione heatmap
    getHeatMap(jandf,metjandf,'Jan',dirpath,count,'2019-01') 
    
    
    
        